In [20]:
import tensorflow as tf
import tensorflow_datasets as tfds

imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling imdb_reviews-train.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling imdb_reviews-test.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling imdb_reviews-unsupervised.tfrecord...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /Users/ryleyun/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [23]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels= []

# numpy array로 바꾼 후 utf-8로 디코딩
for s, l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())

for s, l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

In [46]:
training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)

In [47]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<oov>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

# testing은 알지 못하는 자료이니 fit_on_text 하면 안 됨!
testing_sequence = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequence, maxlen = max_length, truncating = trunc_type)

In [48]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

decode_review(padded[3])

'? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <oov> into a big arm chair and <oov> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <oov> to cross no dangerous waters just a warm and witty <oov> through new york life at its best a family film in every sense and one that deserves the praise it received'

In [49]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                           tf.keras.layers.Conv1D(128, 5, activation='relu'),
                           tf.keras.layers.GlobalAveragePooling1D(),
                            tf.keras.layers.Dense(64, activation='relu'),
                            tf.keras.layers.Dense(1, activation='sigmoid')])

In [51]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['acc'])

In [55]:
NUM_EPOCHS = 10

history = model.fit(padded, training_labels, validation_data=(testing_padded, testing_labels),
                    epochs=NUM_EPOCHS)

Epoch 1/10
782/782 [==============================] - 3s 4ms/step - loss: 0.2237 - acc: 0.9134 - val_loss: 0.4409 - val_acc: 0.8175
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.1820 - acc: 0.9338 - val_loss: 0.4873 - val_acc: 0.8120
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.1448 - acc: 0.9492 - val_loss: 0.5304 - val_acc: 0.8077
Epoch 4/10
782/782 [==============================] - 2s 3ms/step - loss: 0.1126 - acc: 0.9622 - val_loss: 0.6543 - val_acc: 0.8041
Epoch 5/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0810 - acc: 0.9759 - val_loss: 0.7278 - val_acc: 0.7996
Epoch 6/10
782/782 [==============================] - 3s 3ms/step - loss: 0.0641 - acc: 0.9804 - val_loss: 0.8552 - val_acc: 0.7973
Epoch 7/10
782/782 [==============================] - 3s 3ms/step - loss: 0.0486 - acc: 0.9849 - val_loss: 1.0439 - val_acc: 0.7949
Epoch 8/10
782/782 [==============================] - 2s 3ms/step - loss: 0.

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel('Epochs')
    plt.ylabel(string)
    plt.legend([string, 'val_' + string])
    plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')
